<a href="https://colab.research.google.com/github/a-mhamdi/jlai/blob/main/Codes/Julia/Part-3/transfer-learning/transfer-learning-mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning (MNIST DATASET)
---

In [ ]:
versioninfo()

Julia Version 1.11.2
Commit 5e9a32e7af2 (2024-12-01 20:02 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 8 × Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz
  WORD_SIZE: 64
  LLVM: libLLVM-16.0.6 (ORCJIT, skylake)
Threads: 1 default, 0 interactive, 1 GC (on 8 virtual cores)
Environment:
  LD_LIBRARY_PATH = /home/mhamdi/torch/install/lib:/home/mhamdi/torch/install/lib:/home/mhamdi/torch/install/lib:
  DYLD_LIBRARY_PATH = /home/mhamdi/torch/install/lib:/home/mhamdi/torch/install/lib:/home/mhamdi/torch/install/lib:
  JULIA_NUM_THREADS = 8


In [ ]:
using Pkg; pkg"activate ."

  Activating project at `~/Work/git-repos/AI-ML-DL/jlai/Codes/Julia/Part-3/transfer-learning`


Status `~/Work/git-repos/AI-ML-DL/jlai/Codes/Julia/Part-3/transfer-learning/Project.toml`
  [88a5189c] DataAugmentation v0.3.2
⌅ [587475ba] Flux v0.14.25
  [916415d5] Images v0.26.1
  [eb30cadb] MLDatasets v0.7.18
  [dbeba491] Metalhead v0.9.4
  [c3e4b0f8] Pluto v0.20.4
  [7f904dfe] PlutoUI v0.7.60
  [d6f4376e] Markdown v1.11.0
Info Packages marked with ⌅ have new versions available but compatibility constraints restrict them from upgrading. To see why use `status --outdated`


In [ ]:
Pkg.add(["DataAugmentation", "Flux", "Images", "ProgressMeter", "MLDatasets", "Metalhead", "CUDA", "cuDNN"])

In [ ]:
pkg"status"

Load the pre-trained model

In [ ]:
using Metalhead

[API Reference](https://fluxml.ai/Metalhead.jl/dev/api/reference/#API-Reference)

In [ ]:
vgg = VGG(16; pretrain=true).layers;

In [ ]:
using DataAugmentation

In [ ]:
tfm = CenterCrop((224, 224)) |> ImageToTensor()

Sequence{Tuple{Crop{2, DataAugmentation.FromCenter}, ImageToTensor{Float32}}}((Crop{2, DataAugmentation.FromCenter}((224, 224), DataAugmentation.FromCenter()), ImageToTensor{Float32}()))

In [ ]:
using Flux
using Flux: onecold, onehotbatch

In [ ]:
model = Chain(
    vgg[1:end-1],
    vgg[end][1:end-1],
    # Replace the last layer
    Dense(4096, 256),
    Dense(256, 10)
    ) |> gpu

Chain(
  Chain(
    Chain(
      Conv((3, 3), 3 => 64, relu, pad=1),  # 1_792 parameters
      Conv((3, 3), 64 => 64, relu, pad=1),  # 36_928 parameters
      MaxPool((2, 2)),
      Conv((3, 3), 64 => 128, relu, pad=1),  # 73_856 parameters
      Conv((3, 3), 128 => 128, relu, pad=1),  # 147_584 parameters
      MaxPool((2, 2)),
      Conv((3, 3), 128 => 256, relu, pad=1),  # 295_168 parameters
      Conv((3, 3), 256 => 256, relu, pad=1),  # 590_080 parameters
      Conv((3, 3), 256 => 256, relu, pad=1),  # 590_080 parameters
      MaxPool((2, 2)),
      Conv((3, 3), 256 => 512, relu, pad=1),  # 1_180_160 parameters
      Conv((3, 3), 512 => 512, relu, pad=1),  # 2_359_808 parameters
      Conv((3, 3), 512 => 512, relu, pad=1),  # 2_359_808 parameters
      MaxPool((2, 2)),
      Conv((3, 3), 512 => 512, relu, pad=1),  # 2_359_808 parameters
      Conv((3, 3), 512 => 512, relu, pad=1),  # 2_359_808 parameters
      Conv((3, 3), 512 => 512, relu, pad=1),  # 2_359_808 parameters
      Ma

In [ ]:
using MLDatasets

Load the MNIST dataset

In [ ]:
function get_data(split)
    data = MNIST(split)
    imgs, y = data.features ./ 255, onehotbatch(data.targets, 0:9);
    X = []
    for i in 1:length(y)
        img = apply(tfm, Image(RGB.(imgs[:,:,i]))) |> itemdata
        push!(X, img)
    end
    loader = Flux.Data.DataLoader((X, y); batchsize=64, shuffle=true) |> gpu;
    return loader
end

get_data (generic function with 1 method)

In [ ]:
using Images

In [ ]:
train_loader = get_data(:train);
test_loader = get_data(:test);

LoadError: UndefVarError: `get_data` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

**Define a loss function and an optimizer**

In [ ]:
loss_fn = Flux.logitcrossentropy
opt_state = Flux.setup(Adam(3e-3), model[end]) # Freeze the weights of the pre-trained layers

In [ ]:
using ProgressMeter

In [ ]:
epochs = 3

Fine-tune the model

In [ ]:
for epoch in 1:epochs
    @showprogress for (X, y) in train_loader
        # Compute the gradient of the loss with respect to the model's parameters
        loss, ∇ = Flux.withgradient(model) do m
            ŷ = m(X)
            loss_fn(ŷ, y)
        end
        # Update the model's parameters using the optimizer
        Flux.update!(opt_state, model, ∇[1])
    end
    @info "Calculate the accuracy on the test set"
    for (X, y) in test_loader
        ys = onecold(y, 0:9) |> cpu
        ŷ  = onecold(model(X), 0:9) |> cpu
        accuracy = sum( ys .== ŷ ) / length(ys)
        println("Epoch: $epoch, Accuracy: $accuracy")
    end
end